In [25]:
import torch
import os
import numpy as np

In [27]:
# List all .pt files in the directory
local_path = '../embeddings/local/info_vae_dim64_reg10_kld1e-3'
global_path = '../embeddings/global_with_ssl/'
local_files = [f for f in os.listdir(local_path) if f.endswith('.pt')]
global_files = [f for f in os.listdir(global_path) if f.endswith('.pt')]

local_names = []
local_arr = []

for file in local_files:
    local_names.append(file)
    tensor = torch.load(os.path.join(local_path, file))
    local_arr.append(tensor.cpu().numpy())

global_names = []
local_arr = np.array(local_arr)

global_arr = []
for file in global_files:
    global_names.append(file[:-3]+"_embedding.pt")
    tensor = torch.load(os.path.join(global_path, file))
    global_arr.append(tensor.cpu().numpy())

global_arr = np.array(global_arr)


C:\Users\PresiJean\AppData\Local\Temp\ipykernel_26512\3260505980.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(os.path.join(local_path, file))
C:\

In [28]:
sorted_indices = np.argsort(global_names)
global_names = [global_names[i] for i in sorted_indices]
global_arr = global_arr[sorted_indices]

sorted_indices = np.argsort(local_names)
local_names = [local_names[i] for i in sorted_indices]
local_arr = local_arr[sorted_indices]

In [29]:
print("Local embeddings shape:", local_arr.shape)
print("Global embeddings shape:", global_arr.shape)

Local embeddings shape: (8739, 320)
Global embeddings shape: (8739, 1024)


In [30]:
N = max(len(local_arr), len(global_arr))
for i in range(N):
    if local_names[i] != global_names[i]:
        raise ValueError(f"Mismatch in file names: {local_names[i]} != {global_names[i]}")

In [31]:
merged_arr = np.zeros((N, local_arr.shape[1]+global_arr.shape[1]))  # Assuming embeddings are of size 128

for i in range(N):
    merged_arr[i, :local_arr.shape[1]] = local_arr[i]
    merged_arr[i, local_arr.shape[1]:] = global_arr[i]

# Save the merged array to a .npy file
output_path = '../embeddings/merged_embeddings.npy'
np.save(output_path, merged_arr)
output_path = '../embeddings/local_embeddings.npy'
np.save(output_path, local_arr)
output_path = '../embeddings/global_embeddings.npy'
np.save(output_path, global_arr)
output_path = '../embeddings/names_embeddings.npy'
np.save(output_path, global_names)